In [1]:
import pandas as pd
from sqlalchemy import create_engine, inspect

### Store CSV into DataFrame

In [2]:
csv_file = "Resources/worldometer_data.csv"
worldometer_df = pd.read_csv(csv_file)
worldometer_df.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
0,USA,North America,3.311981e+08,5032179,NaN,162804.0,NaN,2576668.0,NaN,2292707.0,18296.0,15194.0,492.0,63139605.0,190640.0,Americas
1,Brazil,South America,2.127107e+08,2917562,NaN,98644.0,NaN,2047660.0,NaN,771258.0,8318.0,13716.0,464.0,13206188.0,62085.0,Americas
2,India,Asia,1.381345e+09,2025409,NaN,41638.0,NaN,1377384.0,NaN,606387.0,8944.0,1466.0,30.0,22149351.0,16035.0,South-EastAsia
3,Russia,Europe,1.459409e+08,871894,NaN,14606.0,NaN,676357.0,NaN,180931.0,2300.0,5974.0,100.0,29716907.0,203623.0,Europe
4,South Africa,Africa,5.938157e+07,538184,NaN,9604.0,NaN,387316.0,NaN,141264.0,539.0,9063.0,162.0,3149807.0,53044.0,Africa


### Create new data with select columns

In [3]:
worldometer_df = worldometer_df[['Country/Region', 'Population', 'TotalCases', 'TotalDeaths', 'Tot Cases/1M pop', 'Tot Cases/1M pop', 'TotalTests', 'Tests/1M pop']].copy()
worldometer_df.head()

,Country/Region,Population,TotalCases,TotalDeaths,Tot Cases/1M pop,Tot Cases/1M pop,TotalTests,Tests/1M pop
0,USA,3.311981e+08,5032179,162804.0,15194.0,15194.0,63139605.0,190640.0
1,Brazil,2.127107e+08,2917562,98644.0,13716.0,13716.0,13206188.0,62085.0
2,India,1.381345e+09,2025409,41638.0,1466.0,1466.0,22149351.0,16035.0
3,Russia,1.459409e+08,871894,14606.0,5974.0,5974.0,29716907.0,203623.0
4,South Africa,5.938157e+07,538184,9604.0,9063.0,9063.0,3149807.0,53044.0


In [4]:
worldometer_df = worldometer_df.fillna(0)
worldometer_df


,Country/Region,Population,TotalCases,TotalDeaths,Tot Cases/1M pop,Tot Cases/1M pop,TotalTests,Tests/1M pop
0,USA,3.311981e+08,5032179,162804.0,15194.0,15194.0,63139605.0,190640.0
1,Brazil,2.127107e+08,2917562,98644.0,13716.0,13716.0,13206188.0,62085.0
2,India,1.381345e+09,2025409,41638.0,1466.0,1466.0,22149351.0,16035.0
3,Russia,1.459409e+08,871894,14606.0,5974.0,5974.0,29716907.0,203623.0
4,South Africa,5.938157e+07,538184,9604.0,9063.0,9063.0,3149807.0,53044.0
...,...,...,...,...,...,...,...,...
204,Montserrat,4.992000e+03,13,1.0,2604.0,2604.0,61.0,12220.0
205,Caribbean Netherlands,2.624700e+04,13,0.0,495.0,495.0,424.0,16154.0
206,Falkland Islands,3.489000e+03,13,0.0,3726.0,3726.0,1816.0,520493.0
207,Vatican City,8.010000e+02,12,0.0,14981.0,14981.0,0.0,0.0


### Connect to local database

In [6]:
connection_string = "postgres:postgres@localhost:5432/covid_db"
engine = create_engine(f'postgresql://{connection_string}')

### Check for tables

In [7]:
inspector = inspect(engine)

In [10]:
inspector.get_table_names()

['world_covid_stats']

### Use pandas to load csv converted DataFrame into database

In [9]:
worldometer_df.to_sql(name='world_covid_stats', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the customer_name table
* NOTE: can also check using pgAdmin

In [11]:
pd.read_sql_query('select * from world_covid_stats', con=engine).head()

,Country/Region,Population,TotalCases,TotalDeaths,Tot Cases/1M pop,TotalTests,Tests/1M pop
0,USA,3.311981e+08,5032179,162804.0,15194.0,63139605.0,190640.0
1,Brazil,2.127107e+08,2917562,98644.0,13716.0,13206188.0,62085.0
2,India,1.381345e+09,2025409,41638.0,1466.0,22149351.0,16035.0
3,Russia,1.459409e+08,871894,14606.0,5974.0,29716907.0,203623.0
4,South Africa,5.938157e+07,538184,9604.0,9063.0,3149807.0,53044.0
